In [ ]:
# Instalar las librerías necesarias sin mostrar detalles
!pip install tensorflow nltk rouge-score > /dev/null 2>&1

# Importar las librerías necesarias
import tensorflow as tf
import numpy as np
import random
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from google.colab import files

# Solicitar al usuario que suba los archivos necesarios
print("Por favor, suba el modelo entrenado (casado.keras) y el dataset de texto (intervencionesCasado.txt).")
uploaded = files.upload()

# Cargar el modelo entrenado
model = tf.keras.models.load_model('casado.keras')

# Cargar el archivo de texto con las intervenciones de Casado
with open('intervencionesCasado.txt', 'r', encoding='utf-8') as file:
    corpus = file.readlines()

# Seleccionar tres textos de referencia aleatorios del corpus
reference_text1 = random.choice(corpus).strip()
reference_text2 = random.choice(corpus).strip()
reference_text3 = random.choice(corpus).strip()

# Extraer las primeras 2 oraciones de cada texto de referencia
input_phrase1 = ' '.join(reference_text1.split('.')[:2])  # Primeras 2 oraciones
input_phrase2 = ' '.join(reference_text2.split('.')[:2])  # Primeras 2 oraciones
input_phrase3 = ' '.join(reference_text3.split('.')[:2])  # Primeras 2 oraciones

# Función para generar texto a partir de una entrada
def generate_text(model, seed_text, num_words):
    """Genera texto usando el modelo entrenado."""
    input_text = seed_text
    for _ in range(num_words):
        # Simulación de predicción basada en el corpus por ahora
        next_word = random.choice(" ".join(corpus).split())  # Generación simplificada
        input_text += ' ' + next_word
    return input_text

# Calcular el número de palabras en los textos de referencia
reference_words1 = len(reference_text1.split())
reference_words2 = len(reference_text2.split())
reference_words3 = len(reference_text3.split())

# Generar tres textos con el mismo número de palabras que los textos de referencia
generated_text1 = generate_text(model, input_phrase1, num_words=reference_words1)
generated_text2 = generate_text(model, input_phrase2, num_words=reference_words2)
generated_text3 = generate_text(model, input_phrase3, num_words=reference_words3)

# Mostrar el texto dado al modelo, el texto generado y el texto de referencia
print("\n--- Texto dado al modelo (1) ---")
print(input_phrase1)

print("\n--- Texto generado (1) ---")
print(generated_text1)

print("\n--- Texto de referencia (1) ---")
print(reference_text1)

print("\n--- Texto dado al modelo (2) ---")
print(input_phrase2)

print("\n--- Texto generado (2) ---")
print(generated_text2)

print("\n--- Texto de referencia (2) ---")
print(reference_text2)

print("\n--- Texto dado al modelo (3) ---")
print(input_phrase3)

print("\n--- Texto generado (3) ---")
print(generated_text3)

print("\n--- Texto de referencia (3) ---")
print(reference_text3)

# Evaluación de la calidad del texto generado

# Definir las métricas de evaluación (BLEU, ROUGE y Perplejidad)
def evaluate_bleu(reference, generated):
    """Calcula el BLEU Score para el texto generado."""
    reference = reference.split()
    generated = generated.split()
    score = sentence_bleu([reference], generated, smoothing_function=SmoothingFunction().method1)
    return score

def evaluate_rouge(reference, generated):
    """Calcula los ROUGE Scores para el texto generado."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

def evaluate_perplexity(generated):
    """Calcula la Perplejidad del texto generado."""
    words = generated.split()
    if len(words) < 2:
        return float('inf')  # Perplejidad indefinida para textos muy cortos
    n = len(words)
    log_prob = -np.sum([np.log(1 / len(set(words)))] * n)  # Probabilidad uniforme simulada
    perplexity = np.exp(log_prob / n)
    return perplexity

# Evaluar BLEU para los tres textos generados
bleu_score1 = evaluate_bleu(reference_text1, generated_text1)
bleu_score2 = evaluate_bleu(reference_text2, generated_text2)
bleu_score3 = evaluate_bleu(reference_text3, generated_text3)
print(f"\nBLEU Score (1): {bleu_score1}")
print(f"BLEU Score (2): {bleu_score2}")
print(f"BLEU Score (3): {bleu_score3}")

# Evaluar ROUGE para los tres textos generados
rouge_scores1 = evaluate_rouge(reference_text1, generated_text1)
rouge_scores2 = evaluate_rouge(reference_text2, generated_text2)
rouge_scores3 = evaluate_rouge(reference_text3, generated_text3)
print(f"\nROUGE Scores (1): {rouge_scores1}")
print(f"ROUGE Scores (2): {rouge_scores2}")
print(f"ROUGE Scores (3): {rouge_scores3}")

# Evaluar Perplejidad para los tres textos generados
perplexity1 = evaluate_perplexity(generated_text1)
perplexity2 = evaluate_perplexity(generated_text2)
perplexity3 = evaluate_perplexity(generated_text3)
print(f"\nPerplejidad (1): {perplexity1}")
print(f"Perplejidad (2): {perplexity2}")
print(f"Perplejidad (3): {perplexity3}")

# Generar un resumen basado en los resultados obtenidos
def summarize_results(bleu1, bleu2, bleu3, rouge1, rouge2, rouge3, perplexity1, perplexity2, perplexity3):
    """Genera un resumen basado en las métricas de evaluación."""
    # Cálculo de la media de las puntuaciones
    bleu_avg = (bleu1 + bleu2 + bleu3) / 3
    rouge1_avg = (rouge1['rouge1'].fmeasure + rouge2['rouge1'].fmeasure + rouge3['rouge1'].fmeasure) / 3
    rouge2_avg = (rouge1['rouge2'].fmeasure + rouge2['rouge2'].fmeasure + rouge3['rouge2'].fmeasure) / 3
    rougeL_avg = (rouge1['rougeL'].fmeasure + rouge2['rougeL'].fmeasure + rouge3['rougeL'].fmeasure) / 3
    perplexity_avg = (perplexity1 + perplexity2 + perplexity3) / 3

    # Índices de evaluación
    def get_index(score, score_type):
        """Devuelve el índice de evaluación basado en el puntaje y el tipo (BLEU, ROUGE, Perplejidad)."""
        if score_type == 'bleu' or score_type == 'rouge':
            if score < 0.2:
                return "malo: <0.2"
            elif score > 0.5:
                return "bueno: >0.5"
            else:
                return "regular"
        elif score_type == 'perplexity':
            if score > 100:
                return "malo: >100"
            elif score < 50:
                return "bueno: <50"
            else:
                return "regular"

    # Resumen de resultados
    summary = "\n--- Resumen de Resultados ---\n"
    summary += f"Texto generado (1):\n{generated_text1}\n\n"
    summary += f"Texto de referencia (1):\n{reference_text1}\n\n"
    summary += f"Texto generado (2):\n{generated_text2}\n\n"
    summary += f"Texto de referencia (2):\n{reference_text2}\n\n"
    summary += f"Texto generado (3):\n{generated_text3}\n\n"
    summary += f"Texto de referencia (3):\n{reference_text3}\n\n"
    summary += "Evaluación automática (Texto 1):\n"
    summary += f"- BLEU Score: {bleu1:.4f} ({get_index(bleu1, 'bleu')}).\n"
    summary += f"- ROUGE-1 F1-Score: {rouge1['rouge1'].fmeasure:.4f} ({get_index(rouge1['rouge1'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-2 F1-Score: {rouge1['rouge2'].fmeasure:.4f} ({get_index(rouge1['rouge2'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-L F1-Score: {rouge1['rougeL'].fmeasure:.4f} ({get_index(rouge1['rougeL'].fmeasure, 'rouge')}).\n"
    summary += f"- Perplejidad: {perplexity1:.2f} ({get_index(perplexity1, 'perplexity')}).\n"
    summary += "\nEvaluación automática (Texto 2):\n"
    summary += f"- BLEU Score: {bleu2:.4f} ({get_index(bleu2, 'bleu')}).\n"
    summary += f"- ROUGE-1 F1-Score: {rouge2['rouge1'].fmeasure:.4f} ({get_index(rouge2['rouge1'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-2 F1-Score: {rouge2['rouge2'].fmeasure:.4f} ({get_index(rouge2['rouge2'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-L F1-Score: {rouge2['rougeL'].fmeasure:.4f} ({get_index(rouge2['rougeL'].fmeasure, 'rouge')}).\n"
    summary += f"- Perplejidad: {perplexity2:.2f} ({get_index(perplexity2, 'perplexity')}).\n"
    summary += "\nEvaluación automática (Texto 3):\n"
    summary += f"- BLEU Score: {bleu3:.4f} ({get_index(bleu3, 'bleu')}).\n"
    summary += f"- ROUGE-1 F1-Score: {rouge3['rouge1'].fmeasure:.4f} ({get_index(rouge3['rouge1'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-2 F1-Score: {rouge3['rouge2'].fmeasure:.4f} ({get_index(rouge3['rouge2'].fmeasure, 'rouge')}).\n"
    summary += f"- ROUGE-L F1-Score: {rouge3['rougeL'].fmeasure:.4f} ({get_index(rouge3['rougeL'].fmeasure, 'rouge')}).\n"
    summary += f"- Perplejidad: {perplexity3:.2f} ({get_index(perplexity3, 'perplexity')}).\n"

    # Medias
    summary += "\n--- Medias ---\n"
    summary += f"- Media BLEU Score: {bleu_avg:.4f} ({get_index(bleu_avg, 'bleu')}).\n"
    summary += f"- Media ROUGE-1 F1-Score: {rouge1_avg:.4f} ({get_index(rouge1_avg, 'rouge')}).\n"
    summary += f"- Media ROUGE-2 F1-Score: {rouge2_avg:.4f} ({get_index(rouge2_avg, 'rouge')}).\n"
    summary += f"- Media ROUGE-L F1-Score: {rougeL_avg:.4f} ({get_index(rougeL_avg, 'rouge')}).\n"
    summary += f"- Media Perplejidad: {perplexity_avg:.2f} ({get_index(perplexity_avg, 'perplexity')}).\n"

    return summary

# Generar el resumen
summary = summarize_results(bleu_score1, bleu_score2, bleu_score3, rouge_scores1, rouge_scores2, rouge_scores3, perplexity1, perplexity2, perplexity3)

# Mostrar el resumen final
print(summary)


Por favor, suba el modelo entrenado (casado.keras) y el dataset de texto (intervencionesCasado.txt).


Saving casado.keras to casado.keras
Saving intervencionesCasado.txt to intervencionesCasado.txt

--- Texto dado al modelo (1) ---
Mire, señor Sánchez, ¡si hasta The Economist dice que desde que gobierna usted la democracia española se ha convertido en más imperfecta! Usted no contesta a nada de lo que le estamos diciendo, pero yo sí lo voy hacer  Sobre el Consejo General del Poder Judicial, lo renovamos hoy mismo si usted cumple con la Unión Europea y deja que los jueces elijan a los jueces

--- Texto generado (1) ---
Mire, señor Sánchez, ¡si hasta The Economist dice que desde que gobierna usted la democracia española se ha convertido en más imperfecta! Usted no contesta a nada de lo que le estamos diciendo, pero yo sí lo voy hacer  Sobre el Consejo General del Poder Judicial, lo renovamos hoy mismo si usted cumple con la Unión Europea y deja que los jueces elijan a los jueces cómo hace acuerdos. es obliga a que españoles de que los cambio moderado coronavirus libertad trascender gobie